In [2]:
import pandas as pd
from pytrends.request import TrendReq
import numpy as np

In [53]:

def tends(keyw, fechaIni = '2019-10-01', fechaFin = '2020-10-01'):
    user_timeframe=fechaIni+' '+fechaFin
    user_cat=7
    #'Finance'
    pytrend = TrendReq(hl='es-PE', tz=360)
    
    pytrend.build_payload(
         kw_list=keyw,
         cat=0,
         timeframe=user_timeframe,
         geo='MX',
         gprop='')
     
    data = pytrend.interest_over_time()
    data= data.drop(labels=['isPartial'],axis='columns').reset_index()
    
    return data


In [54]:
def maxTendencia(wordsTot):
    keywords = wordsTot

    listWord=[]
    for i in keywords:
        listWord.append(i)
        # print(f'listword: {listWord}')
        data0 =tends(keyw=listWord)
        df0 =pd.DataFrame(data0[listWord].max()==100,columns=['bool']).reset_index()
        varMax=list(df0['index'][df0['bool']==True])
        if i==varMax[0]: listWord=varMax
        else: listWord.remove(i)
        # print(listword)
    # print(listWord)
    return listWord, data0


In [55]:
def obtenerTendencia(listWord, keywords):
    key0 = listWord
    #dataf = data0[['date']]
    dataf = pd.DataFrame()
    for i in keywords:
        if i!=key0[0]: 
            key0.append(i)
            print(key0)
            data1 = tends(key0)

            if not dataf.empty:
                listVarSel=list(set(data1.columns) - set(dataf['variable'].unique()))
                data1 = data1[listVarSel]

            colmelt=list(set(data1.columns)-{'date'})
            dataf = dataf.append(data1.melt(id_vars='date', value_vars=colmelt, value_name='valor'))

            #dataf = dataf.drop_duplicates()
            #data1 = data1[[i]]
            #dataf = pd.concat([dataf,data1],axis=1)
            key0.remove(i)

    return dataf

In [78]:

def banc(x):
    ba=""
    if 'santander' in x: ba='SANTANDER PR'
    elif 'bbva' in x or 'bancomer' in x: ba='BANCOMER'
    elif 'banorte' in x: ba='BANORTE'
    elif 'hsbc' in x: ba='HSBC PR'
    elif 'banamex' in x or 'citi' in x: ba='BANAMEX'
    else: ba='OTRO'

    return ba

def codBase(dataf, etiqueta):

    #colsDataf = list(set(dataf.columns)-{'date'})
    #dataf2 = dataf.melt(id_vars='date',value_vars=colsDataf,value_name='valor')

    #dataf2.loc[dataf2['variable'].str.contains('santander'),'banco']='SANTANDER PR'

    # dataf['banco'] = dataf['variable'].map(banc)
    dataf['sector'] = dataf['variable']
    
    dataf3 = dataf.groupby(['sector','date']).agg(sm=('valor','sum')).reset_index()
    dataf3['fh_corte'] = pd.to_datetime(dataf3['date'].dt.strftime('%Y-%m-'+'01'),format='%Y-%m-%d')
    
    dataf4 = dataf3.groupby(['sector']).agg(mn=('sm','mean')).reset_index()
    dataf4['tendencia'] = dataf4['mn']/dataf4['mn'].max()
    #dataf4['tendencia'] = dataf4['mn']/dataf4.groupby('fh_corte').mn.transform(np.sum)
    
    dataf4['etiqueta'] = etiqueta
    
    return dataf4


In [57]:
# Consumo

bancos = ['bancomer','bbva','bbva bancomer','santander','banamex','citibanamex', 'citi', 'banorte','hsbc',
          'scotiabank','banco azteca','bancoppel'
         ]

words=['prestamo en bancomer','prestamo bancomer', 'prestamos bancomer','bancomer prestamo nomina','prestamo personal bancomer', 
       'bancomer prestamo de nomina', 'bancomer prestamo personal', 'solicitar prestamo bancomer', 'bancomer prestamo']
wordsTotConsumo = []
for i in bancos:
    for j in words:
        wordsTotConsumo.append(j.replace('bancomer',i))
        
        
#Tarjetas tdc

words=['bancomer solicitar tarjeta de credito','bancomer tarjeta de credito','solicitar tarjeta de credito bancomer','solicitud tarjeta de credito bancomer',
'tarjeta bancomer','tarjeta de credito bancomer','tarjeta de crédito bancomer','tarjeta de credito bancomer sin anualidad','tarjetas bancomer',
'tarjetas de credito bancomer','tramitar tarjeta de credito bancomer','bancomer tarjeta de crédito','tdc bancomer','bancomer tdc']
wordsTotTdc = []
for i in bancos:
    for j in words:
        wordsTotTdc.append(j.replace('bancomer',i))       

#nómina

words=['bancomer nomina','bancomer nómina','bancomer portabilidad de nomina','portabilidad de nómina bancomer','portabilidad de nomina bancomer','bancomer portabilidad de nómina','bancomer portabilidad nomina','bancomer portabilidad nómina',
      'bancomer cambio de nómina','cambio de nómina bancomer','cambio nómina bancomer']
wordsTotNom = []
for i in bancos:
    for j in words:
        wordsTotNom.append(j.replace('bancomer',i)) 
    
#vista

words=['cuenta bancomer','cuenta de ahorro bancomer','bancomer cuenta de ahorro','cuenta ahorro bancomer',
       'cuenta débito bancomer','cuenta debito bancomer','bancomer cuenta debito','cuenta de debito bancomer',
      'bancomer cuenta depósito','bancomer cuenta deposito','cuenta deposito bancomer','cuenta depósito bancomer','cuenta de deposito bancomer',
       'cuenta de depósito bancomer']
wordsTotVist = []
for i in bancos:
    for j in words:
        wordsTotVist.append(j.replace('bancomer',i))     
    
    

In [58]:
# ruta='/Users/mb66483/Documents/'
# fechaIni='2017-01-01'
# fechaFin='2020-10-01'
# wordsTot = 'wordsTotVist'
#wordsTotConsumo #wordsTotTdc #wordsTotNom #wordsTotVist

# if wordsTot=='wordsTotConsumo': 
#     wordVec = wordsTotConsumo
#     etiqueta='goog_tend_consumo'
# elif wordsTot=='wordsTotTdc': 
#     wordVec = wordsTotTdc
#     etiqueta='goog_tend_tdc'
# elif wordsTot=='wordsTotNom': 
#     wordVec = wordsTotNom
#     etiqueta='goog_tend_nom'
# elif wordsTot=='wordsTotVist': 
#     wordVec = wordsTotVist
#     etiqueta='goog_tend_vista'

wordVec = ['COMERCIALIZADORAS',
 'CONSULTORIAS Y SERVICIOS',
 'CONSTRUCTORAS',
 'TRANSPORTE',
 'METALMECANICA',
 'AGROINDUSTRIA Y GANADERIA',
 'INMOBILIARIAS',
 'TURISMO',
 'OTROS SERVICIOS',
 'HIDROCARBUROS',
 'INFORMATICA',
 'VEHICULOS',
 'TEXTILES',
 'MINERIA',
 'INDUSTRIA ALIMENTARIA',
 'EDUCACION',
 'CAUCHO Y PLASTICO',
 'TELECOMUNICACIONES',
 'PESCA',
 'SALUD',
 'ALQUILERES DE MAQUINARIAS',
 'INTERMEDIACION FINANCIERA',
 'ESPARCIMIENTO',
 'AUTOMOTRIZ',
 'ABOGADOS',
 'QUIMICAS',
 'IMPRESION',
 'MINERALES NO METALICOS',
 'FORESTAL Y SILVICULTURA',
 'ORGANIZACIONES',
 'ENERGIA Y AGUA',
 'EDITORIALES',
 'ADMINISTRACION PUBLICA',
 'CURTIDURIA',
 'PAPEL',
 'CORREOS',
 'RECICLAJE',
 'LABORATORIOS',
 'INVESTIGACION Y DESARROLLO',
 'BEBIDAS',
 'VETERINARIAS']

listWord, data0= maxTendencia(wordVec)
print(f'listword  {listword}')
print(f'data {data0}')


listword  ['Tecnologia']
data          date  SALUD  VETERINARIAS
0  2019-10-06     71             1
1  2019-10-13     68             1
2  2019-10-20     63             1
3  2019-10-27     54             1
4  2019-11-03     60             1
5  2019-11-10     56             1
6  2019-11-17     53             1
7  2019-11-24     51             1
8  2019-12-01     48             1
9  2019-12-08     44             1
10 2019-12-15     35             1
11 2019-12-22     28             2
12 2019-12-29     33             1
13 2020-01-05     47             1
14 2020-01-12     63             1
15 2020-01-19     57             1
16 2020-01-26     59             1
17 2020-02-02     61             1
18 2020-02-09     60             1
19 2020-02-16     60             1
20 2020-02-23     62             1
21 2020-03-01     60             1
22 2020-03-08     65             1
23 2020-03-15     91             1
24 2020-03-22     88             1
25 2020-03-29     91             1
26 2020-04-05     72     

In [67]:
datafl = obtenerTendencia(listWord,wordVec)



['SALUD', 'COMERCIALIZADORAS']
['SALUD', 'CONSULTORIAS Y SERVICIOS']
['SALUD', 'CONSTRUCTORAS']
['SALUD', 'TRANSPORTE']
['SALUD', 'METALMECANICA']
['SALUD', 'AGROINDUSTRIA Y GANADERIA']
['SALUD', 'INMOBILIARIAS']
['SALUD', 'TURISMO']
['SALUD', 'OTROS SERVICIOS']
['SALUD', 'HIDROCARBUROS']
['SALUD', 'INFORMATICA']
['SALUD', 'VEHICULOS']
['SALUD', 'TEXTILES']
['SALUD', 'MINERIA']
['SALUD', 'INDUSTRIA ALIMENTARIA']
['SALUD', 'EDUCACION']
['SALUD', 'CAUCHO Y PLASTICO']
['SALUD', 'TELECOMUNICACIONES']
['SALUD', 'PESCA']
['SALUD', 'ALQUILERES DE MAQUINARIAS']
['SALUD', 'INTERMEDIACION FINANCIERA']
['SALUD', 'ESPARCIMIENTO']
['SALUD', 'AUTOMOTRIZ']
['SALUD', 'ABOGADOS']
['SALUD', 'QUIMICAS']
['SALUD', 'IMPRESION']
['SALUD', 'MINERALES NO METALICOS']
['SALUD', 'FORESTAL Y SILVICULTURA']
['SALUD', 'ORGANIZACIONES']
['SALUD', 'ENERGIA Y AGUA']
['SALUD', 'EDITORIALES']
['SALUD', 'ADMINISTRACION PUBLICA']
['SALUD', 'CURTIDURIA']
['SALUD', 'PAPEL']
['SALUD', 'CORREOS']
['SALUD', 'RECICLAJE']
['SALU

In [74]:
datafl.sample(20)


,date,variable,valor
43,2020-08-02,CURTIDURIA,0
36,2020-06-14,ADMINISTRACION PUBLICA,1
28,2020-04-19,QUIMICAS,5
51,2020-09-27,CONSTRUCTORAS,0
5,2019-11-10,INTERMEDIACION FINANCIERA,0
48,2020-09-06,CAUCHO Y PLASTICO,0
22,2020-03-08,TELECOMUNICACIONES,1
6,2019-11-17,CAUCHO Y PLASTICO,0
22,2020-03-08,EDITORIALES,0
25,2020-03-29,VETERINARIAS,1


In [79]:
data4 = codBase(datafl,'sector_economico')

In [80]:
data4

,sector,mn,tendencia,etiqueta
0,ABOGADOS,3.173077,0.047660,sector_economico
1,ADMINISTRACION PUBLICA,1.019231,0.015309,sector_economico
2,AGROINDUSTRIA Y GANADERIA,0.000000,0.000000,sector_economico
3,ALQUILERES DE MAQUINARIAS,0.000000,0.000000,sector_economico
4,AUTOMOTRIZ,8.788462,0.132005,sector_economico
5,BEBIDAS,6.480769,0.097343,sector_economico
6,CAUCHO Y PLASTICO,0.000000,0.000000,sector_economico
7,COMERCIALIZADORAS,0.000000,0.000000,sector_economico
8,CONSTRUCTORAS,0.788462,0.011843,sector_economico
9,CONSULTORIAS Y SERVICIOS,0.000000,0.000000,sector_economico


In [ ]:
# keywords = ["Agricultura", 'Ganaderia', 'Tecnologia']
# fechaIni='2019-10-15'
# fechaFin='2020-10-15'
# # data = tends(fechaIni, fechaFin, keywords)

# listword, data = maxTendencia(keywords)

In [8]:
#dataf5 = data4.pivot_table(index=['fh_corte'], columns='banco',values='tendencia').reset_index().rename_axis(None, axis=1)

In [9]:
#dataf5.to_excel('/Users/mb66483/Documents/exportprest.xlsx')